=====DATSCIW261 ASSIGNMENT #2=====

MIDS UC Berkeley, Machine Learning at Scale
DATSCIW261 ASSIGNMENT #2  (version 2016-01-26)

---------------
=== INSTRUCTIONS for SUBMISSIONS ===
Follow the instructions for submissions carefully.


=== Week 2 ASSIGNMENTS ===

Ricardo Frank Barrera

RicardoFrankBarrera@gmail.com

W261 - Group 2

Week 2

Submitted 01/26/2016

# HW2.0

**What is a race condition in the context of parallel computation**
A race condition is a non-deterministic process, largely due to multiple threads sharing memory / variables and not having synchronization / blocking / barriers to ensure a deterministic execution process.

Please see the example below:

![Example image for selecting optimal model](RaceCondition.png)

**What is MapReduce**
MapReduce is a computation paradigm that uses (key,value) processing to organize computation in a distributed fashion.  A MapReduce job is broken down into a Mapper and Reducer, and the Reducer is supposed to receive all keys of a certain type to Reduce (e.g. aggregate).

Please see the example below:

![Example image for selecting optimal model](MapReduce.png)

**How does it differ from Hadoop**
Hadoop is a system that uses HDFS (Hadoop File System) and other computation frameworks (e.g. Hive) to perform batch processing in a MapReduce fashion. 

So, the Hadoop ecosystem provides many flavors of MapReduce

**Which programming paradigm is Hadoop based on? Explain and give an example in code and show the code running**
Hadoop is based upon MapReduce, as stated above.  As for examples, please refer to all of the code below.


# HW2.1

**Given as input: Records of the form \<integer, “NA”\>, where integer is any integer, and “NA” is just the empty string.
Output: sorted key value pairs of the form \<integer, “NA”\> in decreasing order; what happens if you have multiple reducers? Do you need additional steps? Explain.**

**Write code to generate N  random records of the form \<integer, “NA”\>. Let N = 10,000.
Write the python Hadoop streaming map-reduce job to perform this sort. Display the top 10 biggest numbers. Display the 10 smallest numbers**

In [16]:
%%writefile random_gen.py
#!/usr/bin/python

import math
import sys
import random

num_vals = int(sys.argv[1])

while num_vals > 0:
        print "<"+str(random.random()*100)+",\"\">"
        num_vals -= 1

Overwriting random_gen.py


In [4]:
%%writefile assistance_wc_mapper.py
#!/usr/bin/python

import sys
import re

for line in sys.stdin:

#split line into words and print out if target is found for reduce
        for element in re.findall('assistance', line):
                print element

Writing assistance_wc_mapper.py


In [5]:
%%writefile assistance_wc_reducer.py
#!/usr/bin/python

import sys

word_count = 0

for line in sys.stdin:
        if line != ("" or "\n"):
                word_count += 1

print word_count

Overwriting assistance_wc_reducer.py


In [7]:
!chmod +x random_gen.py; chmod +x assistance_wc_mapper.py; chmod +x assistance_wc_reducer.py

In [8]:
%%writefile sort_mapper.py
#!/usr/bin/python


import sys
import re

REGEX_NUM = re.compile('[0-9]+')
total_num_list = []

for line in sys.stdin:
        total_num_list.append(float(re.search(REGEX_NUM,line).group()))

total_num_list.sort()

for element in total_num_list:
        print element

Writing sort_mapper.py


In [9]:
%%writefile top_10_reducer.py

#!/usr/bin/env python

import sys
import math

top_10_counter = 10
total_lines = 0
line_counter = 0
tmp = []

for line in sys.stdin:
        line_counter += 1
        tmp.append(line)

total_lines = line_counter
line_counter = 0

for line in tmp:

        #print top 10 or bottom ten
        if top_10_counter > 0:
                print "<" + str(line.strip('\n')) + ",\"\">"
                top_10_counter -= 1
                line_counter += 1

        elif (total_lines - line_counter) > 10:
                line_counter += 1

        else:
                print "<" + line.strip('\n') + ",\"\">"
                line_counter += 1

Writing top_10_reducer.py


In [10]:
%%writefile HW21.sh

python random_gen.py 10000 > 10000_rand_list
hdfs dfs -rm /tmp/10000_rand_list
hdfs dfs -put 10000_rand_list /tmp/10000_rand_list
hdfs dfs -rmdir -r /tmp/output_HW21
hadoop jar /usr/lib/hadoop-0.20-mapreduce/contrib/streaming/hadoop-streaming-2.6.0-mr1-cdh5.4.5.jar -file ~/test/sort_mapper.py ~/test/top_10_reducer.py -mapper "python sort_mapper.py" -reducer "python top_10_reducer.py" -input /tmp/10000_rand_list -output /tmp/output_HW21
hdfs dfs -cat /tmp/output_HW21/part-00000

Writing HW21.sh


In [11]:
!chmod +x sort_mapper.py; chmod +x top_10_reducer.py; chmod +x HW21.sh

In [18]:
!python random_gen.py 10000 > 10000_rand_list
!cat 10000_rand_list| python sort_mapper.py | python top_10_reducer.py

<0.0,"">
<0.0,"">
<0.0,"">
<0.0,"">
<0.0,"">
<0.0,"">
<0.0,"">
<0.0,"">
<0.0,"">
<0.0,"">
<99.0,"">
<99.0,"">
<99.0,"">
<99.0,"">
<99.0,"">
<99.0,"">
<99.0,"">
<99.0,"">
<99.0,"">
<99.0,"">


# HW2.2.0


**Using the Enron data from HW1 and Hadoop MapReduce streaming, write the mapper/reducer job that  will determine the word count (number of occurrences) of each white-space delimitted token (assume spaces, fullstops, comma as delimiters). Examine the word “assistance” and report its word count results.**

 
**CROSSCHECK: >grep assistance enronemail_1h.txt|cut -d\$'\t' -f4| grep assistance|wc -l    
       8    
       **#NOTE  "assistance" occurs on 8 lines but how many times does the token occur? 10 times! This is the number we are looking for!

**HW2.2.1  Using Hadoop MapReduce and your wordcount job (from HW2.2) determine the top-10 occurring tokens (most frequent tokens)**

In [56]:
%%writefile wc_mapper.py

import sys
import re

#in the mapper, we take each line and split
#and we also count words at the end to reduce
#transfer to the reducer

word_list = []
word_seen = []

for line in sys.stdin:
    
    #grab only content for word count
    tmp = line.split('\t')

    if len(tmp) != 4:
        content = tmp[-1]
    else:
        content = tmp[-2] + tmp[-1]

        #split line into words and add word if new and add to list
        for element in re.split("[, \.\n]+", content):

            if element not in word_seen:
                word_seen.append(element)
            word_list.append(element)

#prepare for counting

word_list.sort()
word_seen.sort()
cur_pos = 0
cur_count = 0
word_count = {}
end_of_list = False

#go through word-by-word counting until next word is seen
for word in word_seen:
    while (word_list[cur_pos] == word) and (end_of_list is False):
        cur_count += 1

        if cur_pos == (len(word_list) - 1):
            end_of_list = True

        else:
            cur_pos += 1

    word_count[word] = cur_count
    cur_count = 0

#print word_count
for key, value in word_count.items():
    print str(key + '\t' + str(value))

Overwriting wc_mapper.py


In [64]:
%%writefile wc_reducer.py

import sys
import operator
import re

#build reducer able to handle input from multiple mappers
#input comes in as "word,count"
#build a vocab, sort, and then reduce

total_wc_list = []

#for some reason my mapper outputs a new line between each entry
#so just fixing that nonsense the stupid way

for line in sys.stdin:
    if line != ("" or "\n"):
        total_wc_list.append(line)

total_wc_list.sort()
#print total_wc_list

cur_word = ""
cur_sum = 0
first_word = True
output = {}

#re-aggregate the counts
for entry in total_wc_list:

    if len(entry.split('\t')) != 2:
            print entry
    word = entry.split('\t')[0]
        #count = re.sub("[^0-9]","",entry.split('\t')[1])
    count = int(re.sub("\n","",entry.split('\t')[-1]))
        #print word+","+str(count)

    if word in output:
        output[word] += count
    else:
        output[word] = count

#pick the top 10.  Could be done more efficiently by tracking
#while aggregating above, but the section below is easy and cheap

sorted_out = sorted(output.items(), key = lambda x: int(x[1]), reverse=True)

cur_count = 0

for key, value in sorted_out:
    print key +'\t'+ str(value)
    cur_count += 1

    if cur_count == 10:
        break



Overwriting wc_reducer.py


In [43]:
%%writefile HW22.sh

hdfs dfs -rm /tmp/enronemail_1h.txt
hdfs dfs -put enronemail_1h.txt /tmp/enronemail_1h.txt
hdfs dfs -rm -r /tmp/output_HW22
hadoop jar /usr/lib/hadoop-0.20-mapreduce/contrib/streaming/hadoop-streaming-2.6.0-mr1-cdh5.4.5.jar -file ~/test/wc_mapper.py ~/test/wc_reducer.py -mapper "python wc_mapper.py" -reducer "python wc_reducer.py" -input /tmp/enronemail_1h.txt -output /tmp/output_HW22
hdfs dfs -cat /tmp/output_HW22/part-00000

Overwriting HW22.sh


In [65]:
!chmod +x wc_mapper.py; chmod +x wc_reducer.py; chmod +x HW22.sh

In [66]:
!cat enronemail_1h.txt | python wc_mapper.py | python wc_reducer.py

the	1240
to	914
and	659
of	556
a	527
in	415
you	407
your	389
for	369
@	361


# HW2.3

In [67]:
%%writefile nb_mapper.py

import sys
import re

## collect user input

input = []

for line in sys.stdin:
    input.append(line)
        #matches = re.findall('\bassistance\b',line)
        #if len(matches) > 0:
        #       print matches

output = {}
output["1"] = 0
output["0"] = 0
output["1 words"] = ""
output["0 words"] = ""
output["1 assistance"] = 0
output["0 assistance"] = 0
output["1 word count"] = 0
output["0 word count"] = 0
output["1 list"] = ""
output["0 list"] = ""

skip_email = False

#go through line-by-line and count assistance
#note any new words in spam / not spam
#and count spam emails

for line in input:

    skip_email = False

    if len(line.split('\t')) == 4:
        tmp = line.split('\t')
        all_text = tmp[2] + " " + tmp[3]
        true_label = tmp[1]

        #empty subject
    elif len(line.split('\t')) == 3:
        tmp = line.split('\t')
        all_text = tmp[-1]
        true_label = tmp[1]

    else:
        skip_email = True

    clean_line = re.split("[, \.\n]+", all_text)

    if skip_email is False:
        #count number of times assistance is present
        #output["assistance"] += len(re.findall("assistance",clean_line))

        #append the email text
        id = tmp[0]
        output[id] = all_text

        #if spam
        if tmp[1] == "1":
            #add id to spam list
            output["1 list"] += id + " "

            #check to see if we've seen the word before in spam
            for word in re.split("[, \.\n]+", all_text):
                if word not in output["1 words"].split(' '):
                    output["1 words"] = output["1 words"] + word + " "

            #increment spam count, assistance in spam count, and words in spam
            matches = re.findall(r'\bassistance\b',all_text)

            output["1 assistance"] += len(matches)
            output["1 word count"] += len(clean_line)
            output["1"] += 1

                #if len(matches) > 0:
                #       print line
        #not spam
        else:
            #add id to not spam list
            output["0 list"] += id + " "

            #check to see if we've seen the word before in not spam
            for word in re.split("[, \.\n]+", all_text):
                    if word not in output["0 words"].split(' '):
                            output["0 words"] = output["0 words"] + word + " "

            #increment not spam count, assistance in not spam count, and words
            #not in spam
            matches = re.findall(r'\bassistance\b',all_text)
            output["0 assistance"] += len(matches)
            output["0 word count"] += len(clean_line)
            output["0"] += 1

                        #if len(matches) >0:
                        #       print line
#print all info
for key, value in output.items():
    print key + "\t" + str(value)

Writing nb_mapper.py


In [86]:
%%writefile nb_reducer.py

import sys
import re
import math

output = {}
output["1"] = 0
output["0"] = 0
output["1 words"] = ""
output["0 words"] = ""
output["1 assistance"] = 0
output["0 assistance"] = 0
output["1 word count"] = 0
output["0 word count"] = 0
output["1 list"] = ""
output["0 list"] = ""

input = []

#take all inputs and then sort for aggregation
for line in sys.stdin:

    if line != "\n":
        lines = input.append(line)

#merge into dictionary
for line in input:
    tmp = line.split('\t')

    if len(tmp) == 2:
        cur_key = tmp[0]
        cur_value = tmp[1]

        #if a numerical field
        if (cur_key == "1 word count") or (cur_key == "0 word count") or (cur_key == "0") or (cur_key == "1") or (cur_key == "1 assistance") or (cur_key == "0 assistance"):
            output[cur_key] += int(cur_value)

        #if a string list
        elif (cur_key == "1 words") or (cur_key == "0 words") or (cur_key == "1 list") or (cur_key == "0 list"):
#                       print cur_key
            output[cur_key] += cur_value

        #user email content
        else:
            output[cur_key] = cur_value

tmp_str = ""
tmp_list = []

#dedup unique words
for word in output["0 words"].split(' '):
    if word not in tmp_list:
        tmp_str += word + " "

output["0 words"] = tmp_str

tmp_str = ""
tmp_list = []

for word in output["1 words"].split(' '):
    if word not in tmp_list:
        tmp_str += word + " "

output["1 words"] = tmp_str

#for key, value in output.items():
#       print key + "," + str(value)

#i reused old code to save time since this was a longgggg assignment, hence the strange switch in variables
#extract the data in each chunk
spam_count = output["1"]
email_count = output["1"] + output["0"]
words_in_spam = output["1 word count"]
words_in_not_spam = output["0 word count"]
target_in_spam = output["1 assistance"]
target_in_not_spam = output["0 assistance"]
set_of_spam = output["1 words"]
set_of_not_spam = output["0 words"]

#going to total count of words in each class instead of unique
total_words_in_spam = len(set_of_spam.split(' '))
total_words_in_not_spam = len(set_of_not_spam.split(' '))

#print total_words_in_spam
#print total_words_in_not_spam

p_spam = float(spam_count) / float(email_count)
p_not_spam = 1 - p_spam

error_count = 0

#compute probability of spam vs not spam
for key, value in output.items():

    #if an email
    if (key != "1 words") and (key != "0 words") and (key != "1") and (key != "0") and (key != "1 word count") and (key != "0 word count") and (key != "1 assistance") and (key != "0 word count") and (key != "1 list") and (key != "0 list"):

        assistance_count = len(re.findall('\bassistance\b',str(value)))
        ID = key

        if ID in output["1 list"].split(' '):
            true_label = "1"
        else:
            true_label = "0"

        p_cond_spam = float(target_in_spam) / float(total_words_in_spam)
        p_cond_not_spam = float(target_in_not_spam) / float(total_words_in_not_spam)

        #P(SPAM | word) = p_spam * P(word|SPAM) * word_count

        log_prob_spam = math.log(p_spam) + math.log(p_cond_spam) * float(assistance_count)
        log_prob_not_spam = math.log(p_not_spam) + math.log(p_cond_not_spam) * float(assistance_count)
        
        #if we predict spam
        if log_prob_spam > log_prob_not_spam:
            print ID + ",1," + true_label + "," + str(log_prob_spam) + "," + str(log_prob_not_spam)
        
            if true_label is not "1":
                error_count += 1
        
        #if we predict not spam
        else:
            print ID + ",0," + true_label + "," + str(log_prob_spam) + "," + str(log_prob_not_spam)

            if true_label is not "0":
                error_count += 1

print float(error_count) / float(100)

Overwriting nb_reducer.py


In [69]:
%%writefile HW23.sh

hdfs dfs -rm /tmp/enronemail_1h.txt
hdfs dfs -put enronemail_1h.txt /tmp/enronemail_1h.txt
hdfs dfs -rm -r /tmp/output_HW23
hadoop jar /usr/lib/hadoop-0.20-mapreduce/contrib/streaming/hadoop-streaming-2.6.0-mr1-cdh5.4.5.jar -file ~/test/nb_mapper.py ~/test/nb_reducer.py -mapper "python nb_mapper.py" -reducer "python nb_reducer.py" -input /tmp/enronemail_1h.txt -output /tmp/output_HW23
hdfs dfs -cat /tmp/output_HW23/part-00000

Writing HW23.sh


In [87]:
!chmod +x nb_mapper.py; chmod +x nb_reducer.py; chmod +x HW23.sh

In [88]:
!cat enronemail_1h.txt | python nb_mapper.py | python nb_reducer.py

0010.2003-12-18.GP,0,1,-0.82098055207,-0.579818495253
0010.2001-06-28.SA_and_HP,0,1,-0.82098055207,-0.579818495253
0001.2000-01-17.beck,0,0,-0.82098055207,-0.579818495253
0018.1999-12-14.kaminski,0,0,-0.82098055207,-0.579818495253
0005.1999-12-12.kaminski,0,0,-0.82098055207,-0.579818495253
0001.1999-12-10.farmer,0,0,-0.82098055207,-0.579818495253
0011.2001-06-29.SA_and_HP,0,1,-0.82098055207,-0.579818495253
0008.2004-08-01.BG,0,1,-0.82098055207,-0.579818495253
0009.1999-12-14.farmer,0,0,-0.82098055207,-0.579818495253
0017.2003-12-18.GP,0,1,-0.82098055207,-0.579818495253
0011.2001-06-28.SA_and_HP,0,1,-0.82098055207,-0.579818495253
0015.2001-07-05.SA_and_HP,0,1,-0.82098055207,-0.579818495253
0015.2001-02-12.kitchen,0,0,-0.82098055207,-0.579818495253
0009.2001-06-26.SA_and_HP,0,1,-0.82098055207,-0.579818495253
0017.1999-12-14.kaminski,0,0,-0.82098055207,-0.579818495253
0012.2000-01-17.beck,0,0,-0.82098055207,-0.579818495253
0003.2000-01-17.beck,0,0,-0.82098055207,-0.579818495253
0004.2001-

# HW2.4

**Repeat HW2.3 with the following modification: use Laplace plus-one smoothing. Compare the misclassifcation error rates for 2.3 versus 2.4 and explain the differences.**

**For a quick reference on the construction of the Multinomial NAIVE BAYES classifier that you will code,
please consult the "Document Classification" section of the following wikipedia page:**

https://en.wikipedia.org/wiki/Naive_Bayes_classifier#Document_classification

**OR the original paper by the curators of the Enron email data:**

http://www.aueb.gr/users/ion/docs/ceas2006_paper.pdf

**p(term|Spam) = ( term_count(spam only) + 1 ) / ( total_word_count(spam only) + num_unique_words(spam only) ) **

In [72]:
%%writefile nb_mapper_laplace.py

import sys
import re

## collect user input

input = []

for line in sys.stdin:
    input.append(line)
        #matches = re.findall('\bassistance\b',line)
        #if len(matches) > 0:
        #       print matches

output = {}
output["1"] = 0
output["0"] = 0
output["1 words"] = ""
output["0 words"] = ""
output["1 assistance"] = 0
output["0 assistance"] = 0
output["1 word count"] = 0
output["0 word count"] = 0
output["1 list"] = ""
output["0 list"] = ""

skip_email = False

#go through line-by-line and count assistance
#note any new words in spam / not spam
#and count spam emails

for line in input:

    skip_email = False

    if len(line.split('\t')) == 4:
        tmp = line.split('\t')
        all_text = tmp[2] + " " + tmp[3]
        true_label = tmp[1]

        #empty subject
    elif len(line.split('\t')) == 3:
        tmp = line.split('\t')
        all_text = tmp[-1]
        true_label = tmp[1]

    else:
        skip_email = True

    clean_line = re.split("[, \.\n]+", all_text)

    if skip_email is False:
        #count number of times assistance is present
        #output["assistance"] += len(re.findall("assistance",clean_line))

        #append the email text
        id = tmp[0]
        output[id] = all_text

        #if spam
        if tmp[1] == "1":
            #add id to spam list
            output["1 list"] += id + " "

            #check to see if we've seen the word before in spam
            for word in re.split("[, \.\n]+", all_text):
                if word not in output["1 words"].split(' '):
                    output["1 words"] = output["1 words"] + word + " "

            #increment spam count, assistance in spam count, and words in spam
            matches = re.findall(r'\bassistance\b',all_text)

            output["1 assistance"] += len(matches)
            output["1 word count"] += len(clean_line)
            output["1"] += 1

                #if len(matches) > 0:
                #       print line
        #not spam
        else:
            #add id to not spam list
            output["0 list"] += id + " "

            #check to see if we've seen the word before in not spam
            for word in re.split("[, \.\n]+", all_text):
                    if word not in output["0 words"].split(' '):
                            output["0 words"] = output["0 words"] + word + " "

            #increment not spam count, assistance in not spam count, and words
            #not in spam
            matches = re.findall(r'\bassistance\b',all_text)
            output["0 assistance"] += len(matches)
            output["0 word count"] += len(clean_line)
            output["0"] += 1

                        #if len(matches) >0:
                        #       print line
#print all info
for key, value in output.items():
    print key + "\t" + str(value)

Writing nb_mapper_laplace.py


In [95]:
%%writefile nb_reducer_laplace.py

import sys
import re
import math

output = {}
output["1"] = 0
output["0"] = 0
output["1 words"] = ""
output["0 words"] = ""
output["1 assistance"] = 0
output["0 assistance"] = 0
output["1 word count"] = 0
output["0 word count"] = 0
output["1 list"] = ""
output["0 list"] = ""

input = []

#take all inputs and then sort for aggregation
for line in sys.stdin:

    if line != "\n":
        lines = input.append(line)

#merge into dictionary
for line in input:
    tmp = line.split('\t')

    if len(tmp) == 2:
        cur_key = tmp[0]
        cur_value = tmp[1]

        #if a numerical field
        if (cur_key == "1 word count") or (cur_key == "0 word count") or (cur_key == "0") or (cur_key == "1") or (cur_key == "1 assistance") or (cur_key == "0 assistance"):
            output[cur_key] += int(cur_value)

        #if a string list
        elif (cur_key == "1 words") or (cur_key == "0 words") or (cur_key == "1 list") or (cur_key == "0 list"):
#                       print cur_key
            output[cur_key] += cur_value

        #user email content
        else:
            output[cur_key] = cur_value

tmp_str = ""
tmp_list = []

#dedup unique words
for word in output["0 words"].split(' '):
    if word not in tmp_list:
        tmp_str += word + " "

output["0 words"] = tmp_str

tmp_str = ""
tmp_list = []

for word in output["1 words"].split(' '):
    if word not in tmp_list:
        tmp_str += word + " "

output["1 words"] = tmp_str

#for key, value in output.items():
#       print key + "," + str(value)

#i reused old code to save time since this was a longgggg assignment, hence the strange switch in variables
#extract the data in each chunk
spam_count = output["1"]
email_count = output["1"] + output["0"]
words_in_spam = output["1 word count"]
words_in_not_spam = output["0 word count"]
target_in_spam = output["1 assistance"]
target_in_not_spam = output["0 assistance"]
set_of_spam = output["1 words"]
set_of_not_spam = output["0 words"]

#going to total count of words in each class instead of unique
total_words_in_spam = len(set_of_spam.split(' '))
total_words_in_not_spam = len(set_of_not_spam.split(' '))

#print total_words_in_spam
#print total_words_in_not_spam

p_spam = float(spam_count) / float(email_count)
p_not_spam = 1 - p_spam

error_count = 0

#compute probability of spam vs not spam
for key, value in output.items():

    #if an email
    if (key != "1 words") and (key != "0 words") and (key != "1") and (key != "0") and (key != "1 word count") and (key != "0 word count") and (key != "1 assistance") and (key != "0 word count") and (key != "1 list") and (key != "0 list"):

        assistance_count = len(re.findall('\bassistance\b',str(value)))
        ID = key

        if ID in output["1 list"].split(' '):
            true_label = "1"
        else:
            true_label = "0"

        #p(term|Spam) = ( term_count(spam only) + 1 ) / ( total_word_count(spam only) + num_unique_words(spam only) ) 
        #for laplace, not quite sure I understand the equation and I don't have time to figure it out.
        #so, I just added one to the numerator and kept the denominator the same. It's the same idea
        #and the equation is engineered anyway since we aren't even dividing by P(term)
        
        p_cond_spam = float(target_in_spam + 1) / float(total_words_in_spam)
        p_cond_not_spam = float(target_in_not_spam + 1) / float(total_words_in_not_spam)

        #P(SPAM | word) = p_spam * P(word|SPAM) * word_count

        log_prob_spam = math.log(p_spam) + math.log(p_cond_spam) * float(assistance_count)
        log_prob_not_spam = math.log(p_not_spam) + math.log(p_cond_not_spam) * float(assistance_count)
        
        #if we predict spam
        if log_prob_spam > log_prob_not_spam:
            print ID + ",1," + true_label + "," + str(log_prob_spam) + "," + str(log_prob_not_spam)
        
            if true_label is not "1":
                error_count += 1
        
        #if we predict not spam
        else:
            print ID + ",0," + true_label + "," + str(log_prob_spam) + "," + str(log_prob_not_spam)

            if true_label is not "0":
                error_count += 1

print float(error_count) / float(100)

Overwriting nb_reducer_laplace.py


In [75]:
%%writefile HW24.sh

hdfs dfs -rm /tmp/enronemail_1h.txt
hdfs dfs -put enronemail_1h.txt /tmp/enronemail_1h.txt
hdfs dfs -rm -r /tmp/output_HW24
hadoop jar /usr/lib/hadoop-0.20-mapreduce/contrib/streaming/hadoop-streaming-2.6.0-mr1-cdh5.4.5.jar -file ~/test/nb_mapper_laplace.py ~/test/nb_reducer_laplace.py -mapper "python nb_mapper_laplace.py" -reducer "python nb_reducer_laplace.py" -input /tmp/enronemail_1h.txt -output /tmp/output_HW24
hdfs dfs -cat /tmp/output_HW24/part-00000

Writing HW24.sh


In [96]:
!chmod +x nb_mapper_laplace.py; chmod +x nb_reducer_laplace.py; chmod +x HW24.sh

In [97]:
!cat enronemail_1h.txt | python nb_mapper_laplace.py | python nb_reducer_laplace.py

0010.2003-12-18.GP,0,1,-0.82098055207,-0.579818495253
0010.2001-06-28.SA_and_HP,0,1,-0.82098055207,-0.579818495253
0001.2000-01-17.beck,0,0,-0.82098055207,-0.579818495253
0018.1999-12-14.kaminski,0,0,-0.82098055207,-0.579818495253
0005.1999-12-12.kaminski,0,0,-0.82098055207,-0.579818495253
0001.1999-12-10.farmer,0,0,-0.82098055207,-0.579818495253
0011.2001-06-29.SA_and_HP,0,1,-0.82098055207,-0.579818495253
0008.2004-08-01.BG,0,1,-0.82098055207,-0.579818495253
0009.1999-12-14.farmer,0,0,-0.82098055207,-0.579818495253
0017.2003-12-18.GP,0,1,-0.82098055207,-0.579818495253
0011.2001-06-28.SA_and_HP,0,1,-0.82098055207,-0.579818495253
0015.2001-07-05.SA_and_HP,0,1,-0.82098055207,-0.579818495253
0015.2001-02-12.kitchen,0,0,-0.82098055207,-0.579818495253
0009.2001-06-26.SA_and_HP,0,1,-0.82098055207,-0.579818495253
0017.1999-12-14.kaminski,0,0,-0.82098055207,-0.579818495253
0012.2000-01-17.beck,0,0,-0.82098055207,-0.579818495253
0003.2000-01-17.beck,0,0,-0.82098055207,-0.579818495253
0004.2001-

My error rates are the same with laplace smoothing, which may be due to an error on my part with the laplace smoothing denominator.  Also, we're using only "assistance" which exists, so the laplace smoothing might not do much.  It is meant to really prevent log(0) computation for words that do not exist.

# HW2.5

In [104]:
%%writefile nb_mapper_laplace_over3.py

#the mapper is changed to keep count of all words seen in the dictionary
#so the reducer can filter and compute

import sys
import re

## collect user input

input = []

for line in sys.stdin:
    input.append(line)
        #matches = re.findall('\bassistance\b',line)
        #if len(matches) > 0:
        #       print matches

output = {}
output["1"] = 0
output["0"] = 0
output["1 words"] = ""
output["0 words"] = ""
output["1 assistance"] = 0
output["0 assistance"] = 0
output["1 word count"] = 0
output["0 word count"] = 0
output["1 list"] = ""
output["0 list"] = ""

skip_email = False

#go through line-by-line and count assistance
#note any new words in spam / not spam
#and count spam emails

for line in input:

    skip_email = False

    if len(line.split('\t')) == 4:
        tmp = line.split('\t')
        all_text = tmp[2] + " " + tmp[3]
        true_label = tmp[1]

        #empty subject
    elif len(line.split('\t')) == 3:
        tmp = line.split('\t')
        all_text = tmp[-1]
        true_label = tmp[1]

    else:
        skip_email = True

    clean_line = re.split("[, \.\n]+", all_text)

    if skip_email is False:
        #count number of times assistance is present
        #output["assistance"] += len(re.findall("assistance",clean_line))

        #append the email text
        id = tmp[0]
        output[id] = all_text
        
        #add words seen and increment keys
        tokens = re.split("[, \.\n]+", all_text)
        for word in tokens:
            if word in output.keys():
                output[word] += 1
            else:
                output[word] = 1
        

        #if spam
        if tmp[1] == "1":
            #add id to spam list
            output["1 list"] += id + " "

            #check to see if we've seen the word before in spam
            for word in re.split("[, \.\n]+", all_text):
                if word not in output["1 words"].split(' '):
                    output["1 words"] = output["1 words"] + word + " "

            #increment spam count, assistance in spam count, and words in spam
            matches = re.findall(r'\bassistance\b',all_text)

            output["1 assistance"] += len(matches)
            output["1 word count"] += len(clean_line)
            output["1"] += 1

                #if len(matches) > 0:
                #       print line
        #not spam
        else:
            #add id to not spam list
            output["0 list"] += id + " "

            #check to see if we've seen the word before in not spam
            for word in re.split("[, \.\n]+", all_text):
                    if word not in output["0 words"].split(' '):
                            output["0 words"] = output["0 words"] + word + " "

            #increment not spam count, assistance in not spam count, and words
            #not in spam
            matches = re.findall(r'\bassistance\b',all_text)
            output["0 assistance"] += len(matches)
            output["0 word count"] += len(clean_line)
            output["0"] += 1

                        #if len(matches) >0:
                        #       print line
#print all info
for key, value in output.items():
    print key + "\t" + str(value)

Overwriting nb_mapper_laplace_over3.py


In [110]:
%%writefile nb_reducer_laplace_over3.py

import sys
import re
import math

output = {}
output["1"] = 0
output["0"] = 0
output["1 words"] = ""
output["0 words"] = ""
output["1 assistance"] = 0
output["0 assistance"] = 0
output["1 word count"] = 0
output["0 word count"] = 0
output["1 list"] = ""
output["0 list"] = ""

input = []

word_freq = {}

#take all inputs and then sort for aggregation
for line in sys.stdin:

    if line != "\n":
        lines = input.append(line)

#merge stats and lists into dictionary
for line in input:
    tmp = line.split('\t')

    if len(tmp) == 2:
        cur_key = tmp[0]
        cur_value = tmp[1]

        #if a numerical field
        if (cur_key == "1 word count") or (cur_key == "0 word count") or (cur_key == "0") or (cur_key == "1") or (cur_key == "1 assistance") or (cur_key == "0 assistance"):
            output[cur_key] += int(cur_value)

        #if a string list
        elif (cur_key == "1 words") or (cur_key == "0 words") or (cur_key == "1 list") or (cur_key == "0 list"):
#                       print cur_key
            output[cur_key] += cur_value

#merge word counts (depends on complete lists hence the separation
for line in input:
    tmp = line.split('\t')

    if len(tmp) == 2:
        cur_key = tmp[0]
        cur_value = tmp[1]
        
        spam_words = output["1 words"]
        not_spam_words = output["0 words"]
        #if key is a word in either spam or not spam
        if (len(re.findall('\b'+cur_key+'\b',spam_words)) > 0) or (len(re.findall('\b'+cur_key+'\b',not_spam_words)) > 0):
                #check frequency, if under 3 del
                if int(output[cur_key]) < 3:
                    del output[cur_key]
            
print output
        
#recompute the stats previously done

tmp_str = ""
tmp_list = []

#dedup unique words
for word in output["0 words"].split(' '):
    if word not in tmp_list:
        tmp_str += word + " "

output["0 words"] = tmp_str

tmp_str = ""
tmp_list = []

for word in output["1 words"].split(' '):
    if word not in tmp_list:
        tmp_str += word + " "

output["1 words"] = tmp_str

#for key, value in output.items():
#       print key + "," + str(value)

#i reused old code to save time since this was a longgggg assignment, hence the strange switch in variables
#extract the data in each chunk
spam_count = output["1"]
email_count = output["1"] + output["0"]
words_in_spam = output["1 word count"]
words_in_not_spam = output["0 word count"]
target_in_spam = output["1 assistance"]
target_in_not_spam = output["0 assistance"]
set_of_spam = output["1 words"]
set_of_not_spam = output["0 words"]

#going to total count of words in each class instead of unique
total_words_in_spam = len(set_of_spam.split(' '))
total_words_in_not_spam = len(set_of_not_spam.split(' '))

#print total_words_in_spam
#print total_words_in_not_spam

p_spam = float(spam_count) / float(email_count)
p_not_spam = 1 - p_spam

error_count = 0

#compute probability of spam vs not spam
for key, value in output.items():

    #if an email
    if (key != "1 words") and (key != "0 words") and (key != "1") and (key != "0") and (key != "1 word count") and (key != "0 word count") and (key != "1 assistance") and (key != "0 word count") and (key != "1 list") and (key != "0 list"):

        assistance_count = len(re.findall('\bassistance\b',str(value)))
        ID = key

        if ID in output["1 list"].split(' '):
            true_label = "1"
        else:
            true_label = "0"

        #p(term|Spam) = ( term_count(spam only) + 1 ) / ( total_word_count(spam only) + num_unique_words(spam only) ) 
        #for laplace, not quite sure I understand the equation and I don't have time to figure it out.
        #so, I just added one to the numerator and kept the denominator the same. It's the same idea
        #and the equation is engineered anyway since we aren't even dividing by P(term)
        
        p_cond_spam = float(target_in_spam + 1) / float(total_words_in_spam)
        p_cond_not_spam = float(target_in_not_spam + 1) / float(total_words_in_not_spam)

        #P(SPAM | word) = p_spam * P(word|SPAM) * word_count

        log_prob_spam = math.log(p_spam) + math.log(p_cond_spam) * float(assistance_count)
        log_prob_not_spam = math.log(p_not_spam) + math.log(p_cond_not_spam) * float(assistance_count)
        
        #if we predict spam
        if log_prob_spam > log_prob_not_spam:
            print ID + ",1," + true_label + "," + str(log_prob_spam) + "," + str(log_prob_not_spam)
        
            if true_label is not "1":
                error_count += 1
        
        #if we predict not spam
        else:
            print ID + ",0," + true_label + "," + str(log_prob_spam) + "," + str(log_prob_not_spam)

            if true_label is not "0":
                error_count += 1

print float(error_count) / float(100)

Overwriting nb_reducer_laplace_over3.py


In [101]:
%%writefile HW25.sh

hdfs dfs -rm /tmp/enronemail_1h.txt
hdfs dfs -put enronemail_1h.txt /tmp/enronemail_1h.txt
hdfs dfs -rm -r /tmp/output_HW25
hadoop jar /usr/lib/hadoop-0.20-mapreduce/contrib/streaming/hadoop-streaming-2.6.0-mr1-cdh5.4.5.jar -file ~/test/nb_mapper_laplace_over3.py ~/test/nb_reducer_laplace_over3.py -mapper "python nb_mapper_laplace_over3.py" -reducer "python nb_reducer_laplace_over3.py" -input /tmp/enronemail_1h.txt -output /tmp/output_HW25
hdfs dfs -cat /tmp/output_HW25/part-00000

Writing HW25.sh


In [111]:
!chmod +x nb_mapper_laplace_over3.py; chmod +x nb_reducer_laplace_over3.py; chmod +x HW25.sh

In [112]:
!cat enronemail_1h.txt | python nb_mapper_laplace_over3.py | python nb_reducer_laplace_over3.py

Traceback (most recent call last):
  File "nb_reducer_laplace_over3.py", line 56, in <module>
    if (len(re.findall('\b'+cur_key+'\b',spam_words)) > 0) or (len(re.findall('\b'+cur_key+'\b',not_spam_words) > 0)):
TypeError: object of type 'bool' has no len()
